In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16, 'figure.figsize': (40, 8), 'font.family': 'serif', 'text.usetex': False, 'pgf.rcfonts': False})

%load_ext autoreload
%autoreload 2

from smolyax import indices, nodes

# Asymptotic memory consumption of the interpolation operator

The memory required for the Smolyak interpolation operator is dominated by the tensors that store the function evaluations of the individual tensor-product interpolants. This notebook compares the minimal size of these tensors with their size under the padding strategy implemented in the `SmolyakBarycentricInterpolator` class. The results below indicate that the memory overhead introduced by the padding strategy grows only by a logarithmic factor. This conclusion holds both for asymptotics in the input dimension $d$ and in the number of interpolation nodes $n$.

In [ ]:
to_GB = 1024**(-3)

def size_tensors(k, t, d) :
    indxs = indices.indexset(k, t)
    size = 0
    for nu in indxs :
        zeta = indices.smolyak_coefficient(k, d, t - np.sum([k[j]*nu_j for j,nu_j in nu]), 0)
        if zeta != 0 :
            size += np.prod([si + 1 for _, si in nu])
    return size * np.dtype(np.float64).itemsize * to_GB

def size_padded_tensors(k, t) :
    size = 0
    n_2_nus, _ = indices.non_zero_indices_and_zetas(k, t)

    for n, nus in n_2_nus.items():
        if n == 0:
            size += 1
            continue

        nn = len(nus)  # number of multi-indices of length n
        sorted_dims = np.empty((nn, n), dtype=int)
        sorted_degs = np.empty((nn, n), dtype=int)

        for i, nu in enumerate(n_2_nus[n]):
            sorted_nu = sorted(nu, key=lambda x: x[1], reverse=True)
            sorted_dims[i], sorted_degs[i] = zip(*sorted_nu)

        tau = list(int(ti) for ti in sorted_degs.max(axis=0))
        size += nn * np.prod([si + 1 for si in tau])
    return size * np.dtype(np.float64).itemsize * to_GB

## Number of interpolation nodes $n \to \infty$

In [ ]:
n_list = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200, 102400]
results_n = np.zeros((len(n_list), 2))

d = 400
node_gen = nodes.Leja(dim=d)
k = [np.log((2+j)/np.log(2)) for j in range(d)]
for i, n in enumerate(n_list) :
    t = indices.find_approximate_threshold(k, n, nested=node_gen.is_nested)
    
    results_n[i][0] = size_tensors(k, t, d)
    results_n[i][1] = size_padded_tensors(k, t)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14,6))
ax.loglog(n_list, results_n[:, 0], marker='o', label='tensors raw')
ax.loglog(n_list, np.array(n_list) * np.log(n_list) * 3e-9, 'k--', label=r'$\sim n {\rm log}(n)$')
ax.loglog(n_list, results_n[:, 1], marker='o', label='tensors padded')
ax.loglog(n_list, np.array(n_list) * np.log(n_list)**3 * 5e-10, 'k:', label=r'$\sim n {\rm log}(n)^3$')
ax.set_xlabel(r'$n$'); ax.grid(); ax.legend(); ax.set_ylabel('memory [GB]')
plt.tight_layout()

## Input dimensionality $d \to \infty$

In [ ]:
d_list = [100, 200, 400, 800, 1600, 3200, 6400, 12800]
results_d = np.zeros((len(d_list), 2))

n = 10000
for i, n in enumerate(d_list) :
    node_gen = nodes.Leja(dim=d)
    k = [np.log((2+j)/np.log(2)) for j in range(d)]
    t = indices.find_approximate_threshold(k, n, nested=node_gen.is_nested)
    
    results_d[i][0] = size_tensors(k, t, d)
    results_d[i][1] = size_padded_tensors(k, t)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14,6))
ax.loglog(d_list, results_d[:, 0], marker='o', label='tensors only')
ax.loglog(d_list, np.array(d_list) * np.log(d_list) * 3e-9, 'k--', label=r'$\sim d {\rm log}(d)$')
ax.loglog(d_list, results_d[:, 1], marker='o', label='interpolator')
ax.loglog(d_list, np.array(d_list) * np.log(d_list)**3 * 4e-10, 'k:', label=r'$\sim d  {\rm log}(d)^3$')
ax.set_xlabel(r'$d$'); ax.grid(); ax.legend(); ax.set_ylabel('memory [GB]')
plt.tight_layout()